In [1]:
# Import Packages
import pandas as pd, numpy as np
from pathlib import Path
import os, sys, glob, re
from collections import OrderedDict
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 80)

# Import Board Summary

In [2]:
inputfilepath = Path("Board Summary.csv")

In [3]:
df = pd.read_csv(inputfilepath, low_memory=False)
df

,Ratios - Wealth Delta,Total - Total Annual Compensation,Annual Report Year,Annual- Equity Linked Options - Intrinsic Options (excercisable),Director Type,Characteristics of Roles - Time on Board,Accumulated Wealth - LTIPS(max),Ratios - Bonus/ (Bonus&Salary),Annual- Equity Linked Options - Total Equity Linked Compensation,Annual- Equity Linked Options - Share Price,Individual Name,Annual- Equity Linked Options - LTIPS(max),Director Experience - Total Number of Private Current Boards,Annual Direct Compensation - Salary,Director Experience - Total Number of Other Current Boards,DirectorID*,Accumulated Wealth - Intrinsic Option,Accumulated Wealth - Estimated Option,Annual Direct Compensation - Total Salary+Bonus,Annual- Equity Linked Options - Shares,Director Experience - Nationality Mix,Index,Ticker,Director Experience - Total Number of Private Boards to Date,CompanyID*,Accumulated Wealth - Liquid Wealth,Annual Direct Compensation - Total Inc. D.C. Pension & Other,Annual- Equity Linked Options - Intrinsic Options (unexercisable),Director Experience - Total Number of Quoted Boards to Date,Ratios - %Change from Last Period,Total Wealth - Total Wealth,Country,Annual Direct Compensation - Bonus,Ratios - Performance/ Total,Director Experience - Number of Qualifications,Director Experience - Age (Yrs),Company Name,Annual- Equity Linked Options - Estimated Options (unexcercisable),Annual- Equity Linked Options - Vesting Date,Director Experience - Avg. Yrs on Other Quoted Boards,Characteristics of Roles - Individual Role,Annual Direct Compensation - D.C Pension,Characteristics of Roles - Director Network Size,Director Experience - Total Number of Other Boards to Date,Annual- Equity Linked Options - Estimated Options (exercisable),Annual Direct Compensation - Other,Accumulated Wealth - Shares,Characteristics of Roles - Time to Retirement,Director Count Totals - Number of Independent NED with past CFO/FD role,ISIN,Ratios - Equity Linked/ Total,Characteristics of Roles - Time in Company,Sector,Director Experience - Total Number of Quoted Current Boards,Characteristics of Roles - Time in Role,Director Experience - Gender,inputfilestem,region,mgmt_type,workbook_name,fragment_number,sheet_name
0,2.0,NaN,Current,NaN,SD,3.2,NaN,NaN,NaN,NaN,Robert (Bob) Archie Jennings,NaN,NaN,NaN,NaN,13237186434,NaN,NaN,0.0,NaN,British,FTSE 250,3IN,2.0,9811997164,188.0,0.0,NaN,2.0,NaN,188.0,Jersey,NaN,NaN,2.0,66.0,3I INFRASTRUCTURE PLC (3i Infrastructure Ltd p...,NaN,NaN,6.1,Independent NED,NaN,1617,NaN,NaN,NaN,188.0,4.0,0.0,JE00BF5FX167,NaN,3.2,Investment Companies,2.0,3.2,M,Europe - Board Summary,Europe,Director,Board Summary,NaN,NaN
1,0.0,NaN,Current,NaN,SD,1.2,NaN,NaN,NaN,NaN,Samantha (Sam) Jane Hoe-Richardson,NaN,2.0,NaN,NaN,13534906790,NaN,NaN,0.0,NaN,British,FTSE 250,3IN,4.0,9811997164,5.0,0.0,NaN,2.0,NaN,5.0,Jersey,NaN,NaN,2.0,50.0,3I INFRASTRUCTURE PLC (3i Infrastructure Ltd p...,NaN,NaN,8.2,Independent NED,NaN,1486,NaN,NaN,NaN,5.0,19.8,0.0,JE00BF5FX167,NaN,1.2,Investment Companies,2.0,1.2,F,Europe - Board Summary,Europe,Director,Board Summary,NaN,NaN
2,1.0,NaN,Current,NaN,SD,8.0,NaN,NaN,NaN,NaN,Paul A Masterton,NaN,3.0,NaN,NaN,13584596849,NaN,NaN,0.0,NaN,NaN,FTSE 250,3IN,5.0,9811997164,100.0,0.0,NaN,1.0,NaN,100.0,Jersey,NaN,NaN,3.0,NaN,3I INFRASTRUCTURE PLC (3i Infrastructure Ltd p...,NaN,NaN,0.0,Senior Independent NED,NaN,1509,NaN,NaN,NaN,100.0,NaN,0.0,JE00BF5FX167,NaN,8.0,Investment Companies,1.0,4.3,M,Europe - Board Summary,Europe,Director,Board Summary,NaN,NaN
3,NaN,NaN,Current,NaN,SD,6.9,NaN,NaN,NaN,NaN,Ian Matthew Lobley,NaN,4.0,NaN,NaN,13665216944,NaN,NaN,0.0,NaN,British,FTSE 250,3IN,20.0,9811997164,NaN,0.0,NaN,1.0,NaN,NaN,Jersey,NaN,NaN,1.0,58.0,3I INFRASTRUCTURE PLC (3i Infrastructure Ltd p...,NaN,NaN,0.0,NED,NaN,1129,NaN,NaN,NaN,NaN,11.8,0.0,JE00BF5FX167,NaN,6.9,Investment Companies,1.0,6.9,M,Europe - Board Summary,Europe,Director,Board Summary,NaN,NaN
4,1.0,NaN,Current,NaN,SD,6.3,NaN,NaN,NaN,NaN,James (Doug) Douglas Bannister,NaN,NaN,NaN,Na

# Process year/month

In [4]:
# Get month and year
df['Month'] = df['Annual Report Year'].str[0:4]
df['Year'] = df['Annual Report Year'].str[4:]
df.head(5)

,Ratios - Wealth Delta,Total - Total Annual Compensation,Annual Report Year,Annual- Equity Linked Options - Intrinsic Options (excercisable),Director Type,Characteristics of Roles - Time on Board,Accumulated Wealth - LTIPS(max),Ratios - Bonus/ (Bonus&Salary),Annual- Equity Linked Options - Total Equity Linked Compensation,Annual- Equity Linked Options - Share Price,Individual Name,Annual- Equity Linked Options - LTIPS(max),Director Experience - Total Number of Private Current Boards,Annual Direct Compensation - Salary,Director Experience - Total Number of Other Current Boards,DirectorID*,Accumulated Wealth - Intrinsic Option,Accumulated Wealth - Estimated Option,Annual Direct Compensation - Total Salary+Bonus,Annual- Equity Linked Options - Shares,Director Experience - Nationality Mix,Index,Ticker,Director Experience - Total Number of Private Boards to Date,CompanyID*,Accumulated Wealth - Liquid Wealth,Annual Direct Compensation - Total Inc. D.C. Pension & Other,Annual- Equity Linked Options - Intrinsic Options (unexercisable),Director Experience - Total Number of Quoted Boards to Date,Ratios - %Change from Last Period,Total Wealth - Total Wealth,Country,Annual Direct Compensation - Bonus,Ratios - Performance/ Total,Director Experience - Number of Qualifications,Director Experience - Age (Yrs),Company Name,Annual- Equity Linked Options - Estimated Options (unexcercisable),Annual- Equity Linked Options - Vesting Date,Director Experience - Avg. Yrs on Other Quoted Boards,Characteristics of Roles - Individual Role,Annual Direct Compensation - D.C Pension,Characteristics of Roles - Director Network Size,Director Experience - Total Number of Other Boards to Date,Annual- Equity Linked Options - Estimated Options (exercisable),Annual Direct Compensation - Other,Accumulated Wealth - Shares,Characteristics of Roles - Time to Retirement,Director Count Totals - Number of Independent NED with past CFO/FD role,ISIN,Ratios - Equity Linked/ Total,Characteristics of Roles - Time in Company,Sector,Director Experience - Total Number of Quoted Current Boards,Characteristics of Roles - Time in Role,Director Experience - Gender,inputfilestem,region,mgmt_type,workbook_name,fragment_number,sheet_name,Month,Year
0,2.0,NaN,Current,NaN,SD,3.2,NaN,NaN,NaN,NaN,Robert (Bob) Archie Jennings,NaN,NaN,NaN,NaN,13237186434,NaN,NaN,0.0,NaN,British,FTSE 250,3IN,2.0,9811997164,188.0,0.0,NaN,2.0,NaN,188.0,Jersey,NaN,NaN,2.0,66.0,3I INFRASTRUCTURE PLC (3i Infrastructure Ltd p...,NaN,NaN,6.1,Independent NED,NaN,1617,NaN,NaN,NaN,188.0,4.0,0.0,JE00BF5FX167,NaN,3.2,Investment Companies,2.0,3.2,M,Europe - Board Summary,Europe,Director,Board Summary,NaN,NaN,Curr,ent
1,0.0,NaN,Current,NaN,SD,1.2,NaN,NaN,NaN,NaN,Samantha (Sam) Jane Hoe-Richardson,NaN,2.0,NaN,NaN,13534906790,NaN,NaN,0.0,NaN,British,FTSE 250,3IN,4.0,9811997164,5.0,0.0,NaN,2.0,NaN,5.0,Jersey,NaN,NaN,2.0,50.0,3I INFRASTRUCTURE PLC (3i Infrastructure Ltd p...,NaN,NaN,8.2,Independent NED,NaN,1486,NaN,NaN,NaN,5.0,19.8,0.0,JE00BF5FX167,NaN,1.2,Investment Companies,2.0,1.2,F,Europe - Board Summary,Europe,Director,Board Summary,NaN,NaN,Curr,ent
2,1.0,NaN,Current,NaN,SD,8.0,NaN,NaN,NaN,NaN,Paul A Masterton,NaN,3.0,NaN,NaN,13584596849,NaN,NaN,0.0,NaN,NaN,FTSE 250,3IN,5.0,9811997164,100.0,0.0,NaN,1.0,NaN,100.0,Jersey,NaN,NaN,3.0,NaN,3I INFRASTRUCTURE PLC (3i Infrastructure Ltd p...,NaN,NaN,0.0,Senior Independent NED,NaN,1509,NaN,NaN,NaN,100.0,NaN,0.0,JE00BF5FX167,NaN,8.0,Investment Companies,1.0,4.3,M,Europe - Board Summary,Europe,Director,Board Summary,NaN,NaN,Curr,ent
3,NaN,NaN,Current,NaN,SD,6.9,NaN,NaN,NaN,NaN,Ian Matthew Lobley,NaN,4.0,NaN,NaN,13665216944,NaN,NaN,0.0,NaN,British,FTSE 250,3IN,20.0,9811997164,NaN,0.0,NaN,1.0,NaN,NaN,Jersey,NaN,NaN,1.0,58.0,3I INFRASTRUCTURE PLC (3i Infrastructure Ltd p...,NaN,NaN,0.0,NED,NaN,1129,NaN,NaN,NaN,NaN,11.8,0.0,JE00BF5FX167,NaN,6.9,Investment Companies,1.0,6.9,M,Europe - Board Summary,Europe,Director,Board Summary,NaN,NaN,Curr,ent
4,1.0,NaN,Current,NaN,SD,6.3,NaN,NaN,NaN,Na

In [5]:
# Get current year
# Replace 'Current' with the latest month (previous month of company + 1 year)
current_year = (df[df['Annual Report Year'] != 'Current'].groupby('CompanyID*')['Year'].max().astype('int') + 1).reset_index()

# Get current month
current_month = df[df['Annual Report Year'] != 'Current'].groupby('CompanyID*')['Month'].first().reset_index()

In [6]:
# Get current year and month
current_year_month = current_year.merge(current_month, on='CompanyID*', how='inner')
current_year_month = current_year_month.rename(columns={'Year':'Current Year', 'Month':'Current Month'})
current_year_month

,CompanyID*,Current Year,Current Month
0,270810,2021,Dec
1,386710,2020,Dec
2,386770,2012,Dec
3,422759,2009,Dec
4,493378,2021,Dec
...,...,...,...
3242,305172912690,2021,Jun
3243,305214912690,2021,Dec
3244,305299712697,2020,Dec
3245,306519112781,2020,Dec


In [7]:
# Merge current year and month back to df
df = df.merge(current_year_month, on=['CompanyID*'], how='inner')
# Redefine year and month
df['Year'] = np.where(df['Annual Report Year'] == 'Current', df['Current Year'], df['Year']).astype('int')
df['Month'] = np.where(df['Annual Report Year'] == 'Current', df['Current Month'], df['Month'])
df

,Ratios - Wealth Delta,Total - Total Annual Compensation,Annual Report Year,Annual- Equity Linked Options - Intrinsic Options (excercisable),Director Type,Characteristics of Roles - Time on Board,Accumulated Wealth - LTIPS(max),Ratios - Bonus/ (Bonus&Salary),Annual- Equity Linked Options - Total Equity Linked Compensation,Annual- Equity Linked Options - Share Price,Individual Name,Annual- Equity Linked Options - LTIPS(max),Director Experience - Total Number of Private Current Boards,Annual Direct Compensation - Salary,Director Experience - Total Number of Other Current Boards,DirectorID*,Accumulated Wealth - Intrinsic Option,Accumulated Wealth - Estimated Option,Annual Direct Compensation - Total Salary+Bonus,Annual- Equity Linked Options - Shares,Director Experience - Nationality Mix,Index,Ticker,Director Experience - Total Number of Private Boards to Date,CompanyID*,Accumulated Wealth - Liquid Wealth,Annual Direct Compensation - Total Inc. D.C. Pension & Other,Annual- Equity Linked Options - Intrinsic Options (unexercisable),Director Experience - Total Number of Quoted Boards to Date,Ratios - %Change from Last Period,Total Wealth - Total Wealth,Country,Annual Direct Compensation - Bonus,Ratios - Performance/ Total,Director Experience - Number of Qualifications,Director Experience - Age (Yrs),Company Name,Annual- Equity Linked Options - Estimated Options (unexcercisable),Annual- Equity Linked Options - Vesting Date,Director Experience - Avg. Yrs on Other Quoted Boards,Characteristics of Roles - Individual Role,Annual Direct Compensation - D.C Pension,Characteristics of Roles - Director Network Size,Director Experience - Total Number of Other Boards to Date,Annual- Equity Linked Options - Estimated Options (exercisable),Annual Direct Compensation - Other,Accumulated Wealth - Shares,Characteristics of Roles - Time to Retirement,Director Count Totals - Number of Independent NED with past CFO/FD role,ISIN,Ratios - Equity Linked/ Total,Characteristics of Roles - Time in Company,Sector,Director Experience - Total Number of Quoted Current Boards,Characteristics of Roles - Time in Role,Director Experience - Gender,inputfilestem,region,mgmt_type,workbook_name,fragment_number,sheet_name,Month,Year,Current Year,Current Month
0,2.0,NaN,Current,NaN,SD,3.2,NaN,NaN,NaN,NaN,Robert (Bob) Archie Jennings,NaN,NaN,NaN,NaN,13237186434,NaN,NaN,0.0,NaN,British,FTSE 250,3IN,2.0,9811997164,188.0,0.0,NaN,2.0,NaN,188.0,Jersey,NaN,NaN,2.0,66.0,3I INFRASTRUCTURE PLC (3i Infrastructure Ltd p...,NaN,NaN,6.1,Independent NED,NaN,1617,NaN,NaN,NaN,188.0,4.0,0.0,JE00BF5FX167,NaN,3.2,Investment Companies,2.0,3.2,M,Europe - Board Summary,Europe,Director,Board Summary,NaN,NaN,Mar,2021,2021,Mar
1,0.0,NaN,Current,NaN,SD,1.2,NaN,NaN,NaN,NaN,Samantha (Sam) Jane Hoe-Richardson,NaN,2.0,NaN,NaN,13534906790,NaN,NaN,0.0,NaN,British,FTSE 250,3IN,4.0,9811997164,5.0,0.0,NaN,2.0,NaN,5.0,Jersey,NaN,NaN,2.0,50.0,3I INFRASTRUCTURE PLC (3i Infrastructure Ltd p...,NaN,NaN,8.2,Independent NED,NaN,1486,NaN,NaN,NaN,5.0,19.8,0.0,JE00BF5FX167,NaN,1.2,Investment Companies,2.0,1.2,F,Europe - Board Summary,Europe,Director,Board Summary,NaN,NaN,Mar,2021,2021,Mar
2,1.0,NaN,Current,NaN,SD,8.0,NaN,NaN,NaN,NaN,Paul A Masterton,NaN,3.0,NaN,NaN,13584596849,NaN,NaN,0.0,NaN,NaN,FTSE 250,3IN,5.0,9811997164,100.0,0.0,NaN,1.0,NaN,100.0,Jersey,NaN,NaN,3.0,NaN,3I INFRASTRUCTURE PLC (3i Infrastructure Ltd p...,NaN,NaN,0.0,Senior Independent NED,NaN,1509,NaN,NaN,NaN,100.0,NaN,0.0,JE00BF5FX167,NaN,8.0,Investment Companies,1.0,4.3,M,Europe - Board Summary,Europe,Director,Board Summary,NaN,NaN,Mar,2021,2021,Mar
3,NaN,NaN,Current,NaN,SD,6.9,NaN,NaN,NaN,NaN,Ian Matthew Lobley,NaN,4.0,NaN,NaN,13665216944,NaN,NaN,0.0,NaN,British,FTSE 250,3IN,20.0,9811997164,NaN,0.0,NaN,1.0,NaN,NaN,Jersey,NaN,NaN,1.0,58.0,3I INFRASTRUCTURE PLC (3i Infrastructure Ltd p...,NaN,NaN,0.0,NED,NaN,1129,NaN,NaN,NaN,NaN,11.8,0.0,JE00BF5FX167,NaN,6.9,Investment Companies,1.0,6.9,M,Europe - Board Summary,Europe,Director,Board Summary,NaN,Na

In [8]:
# Only only get data from 2002 onwards
df = df[df['Year'] >= 2002]
df

,Ratios - Wealth Delta,Total - Total Annual Compensation,Annual Report Year,Annual- Equity Linked Options - Intrinsic Options (excercisable),Director Type,Characteristics of Roles - Time on Board,Accumulated Wealth - LTIPS(max),Ratios - Bonus/ (Bonus&Salary),Annual- Equity Linked Options - Total Equity Linked Compensation,Annual- Equity Linked Options - Share Price,Individual Name,Annual- Equity Linked Options - LTIPS(max),Director Experience - Total Number of Private Current Boards,Annual Direct Compensation - Salary,Director Experience - Total Number of Other Current Boards,DirectorID*,Accumulated Wealth - Intrinsic Option,Accumulated Wealth - Estimated Option,Annual Direct Compensation - Total Salary+Bonus,Annual- Equity Linked Options - Shares,Director Experience - Nationality Mix,Index,Ticker,Director Experience - Total Number of Private Boards to Date,CompanyID*,Accumulated Wealth - Liquid Wealth,Annual Direct Compensation - Total Inc. D.C. Pension & Other,Annual- Equity Linked Options - Intrinsic Options (unexercisable),Director Experience - Total Number of Quoted Boards to Date,Ratios - %Change from Last Period,Total Wealth - Total Wealth,Country,Annual Direct Compensation - Bonus,Ratios - Performance/ Total,Director Experience - Number of Qualifications,Director Experience - Age (Yrs),Company Name,Annual- Equity Linked Options - Estimated Options (unexcercisable),Annual- Equity Linked Options - Vesting Date,Director Experience - Avg. Yrs on Other Quoted Boards,Characteristics of Roles - Individual Role,Annual Direct Compensation - D.C Pension,Characteristics of Roles - Director Network Size,Director Experience - Total Number of Other Boards to Date,Annual- Equity Linked Options - Estimated Options (exercisable),Annual Direct Compensation - Other,Accumulated Wealth - Shares,Characteristics of Roles - Time to Retirement,Director Count Totals - Number of Independent NED with past CFO/FD role,ISIN,Ratios - Equity Linked/ Total,Characteristics of Roles - Time in Company,Sector,Director Experience - Total Number of Quoted Current Boards,Characteristics of Roles - Time in Role,Director Experience - Gender,inputfilestem,region,mgmt_type,workbook_name,fragment_number,sheet_name,Month,Year,Current Year,Current Month
0,2.0,NaN,Current,NaN,SD,3.2,NaN,NaN,NaN,NaN,Robert (Bob) Archie Jennings,NaN,NaN,NaN,NaN,13237186434,NaN,NaN,0.0,NaN,British,FTSE 250,3IN,2.0,9811997164,188.0,0.0,NaN,2.0,NaN,188.0,Jersey,NaN,NaN,2.0,66.0,3I INFRASTRUCTURE PLC (3i Infrastructure Ltd p...,NaN,NaN,6.1,Independent NED,NaN,1617,NaN,NaN,NaN,188.0,4.0,0.0,JE00BF5FX167,NaN,3.2,Investment Companies,2.0,3.2,M,Europe - Board Summary,Europe,Director,Board Summary,NaN,NaN,Mar,2021,2021,Mar
1,0.0,NaN,Current,NaN,SD,1.2,NaN,NaN,NaN,NaN,Samantha (Sam) Jane Hoe-Richardson,NaN,2.0,NaN,NaN,13534906790,NaN,NaN,0.0,NaN,British,FTSE 250,3IN,4.0,9811997164,5.0,0.0,NaN,2.0,NaN,5.0,Jersey,NaN,NaN,2.0,50.0,3I INFRASTRUCTURE PLC (3i Infrastructure Ltd p...,NaN,NaN,8.2,Independent NED,NaN,1486,NaN,NaN,NaN,5.0,19.8,0.0,JE00BF5FX167,NaN,1.2,Investment Companies,2.0,1.2,F,Europe - Board Summary,Europe,Director,Board Summary,NaN,NaN,Mar,2021,2021,Mar
2,1.0,NaN,Current,NaN,SD,8.0,NaN,NaN,NaN,NaN,Paul A Masterton,NaN,3.0,NaN,NaN,13584596849,NaN,NaN,0.0,NaN,NaN,FTSE 250,3IN,5.0,9811997164,100.0,0.0,NaN,1.0,NaN,100.0,Jersey,NaN,NaN,3.0,NaN,3I INFRASTRUCTURE PLC (3i Infrastructure Ltd p...,NaN,NaN,0.0,Senior Independent NED,NaN,1509,NaN,NaN,NaN,100.0,NaN,0.0,JE00BF5FX167,NaN,8.0,Investment Companies,1.0,4.3,M,Europe - Board Summary,Europe,Director,Board Summary,NaN,NaN,Mar,2021,2021,Mar
3,NaN,NaN,Current,NaN,SD,6.9,NaN,NaN,NaN,NaN,Ian Matthew Lobley,NaN,4.0,NaN,NaN,13665216944,NaN,NaN,0.0,NaN,British,FTSE 250,3IN,20.0,9811997164,NaN,0.0,NaN,1.0,NaN,NaN,Jersey,NaN,NaN,1.0,58.0,3I INFRASTRUCTURE PLC (3i Infrastructure Ltd p...,NaN,NaN,0.0,NED,NaN,1129,NaN,NaN,NaN,NaN,11.8,0.0,JE00BF5FX167,NaN,6.9,Investment Companies,1.0,6.9,M,Europe - Board Summary,Europe,Director,Board Summary,NaN,Na

# Change data types

In [9]:
# Change data types to enable .groupby to work later on
data_types_dict = {'Year': 'int64'}
df = df.astype(data_types_dict)
df.dtypes

Ratios - Wealth Delta                                                      float64
Total - Total Annual Compensation                                          float64
Annual Report Year                                                          object
Annual- Equity Linked Options - Intrinsic Options (excercisable)           float64
Director Type                                                               object
Characteristics of Roles - Time on Board                                   float64
Accumulated Wealth - LTIPS(max)                                            float64
Ratios - Bonus/ (Bonus&Salary)                                             float64
Annual- Equity Linked Options - Total Equity Linked Compensation           float64
Annual- Equity Linked Options - Share Price                                float64
Individual Name                                                             object
Annual- Equity Linked Options - LTIPS(max)                                 float64
Dire

# Drop duplicates

In [10]:
# Prepare dataset to get variables
# First we get dataset to be at (year, company, director) level. We will later aggregate across directors to get to the (year, company) level.
df = df.drop_duplicates(subset=['Year', 'CompanyID*', 'DirectorID*'], ignore_index=True)
df

,Ratios - Wealth Delta,Total - Total Annual Compensation,Annual Report Year,Annual- Equity Linked Options - Intrinsic Options (excercisable),Director Type,Characteristics of Roles - Time on Board,Accumulated Wealth - LTIPS(max),Ratios - Bonus/ (Bonus&Salary),Annual- Equity Linked Options - Total Equity Linked Compensation,Annual- Equity Linked Options - Share Price,Individual Name,Annual- Equity Linked Options - LTIPS(max),Director Experience - Total Number of Private Current Boards,Annual Direct Compensation - Salary,Director Experience - Total Number of Other Current Boards,DirectorID*,Accumulated Wealth - Intrinsic Option,Accumulated Wealth - Estimated Option,Annual Direct Compensation - Total Salary+Bonus,Annual- Equity Linked Options - Shares,Director Experience - Nationality Mix,Index,Ticker,Director Experience - Total Number of Private Boards to Date,CompanyID*,Accumulated Wealth - Liquid Wealth,Annual Direct Compensation - Total Inc. D.C. Pension & Other,Annual- Equity Linked Options - Intrinsic Options (unexercisable),Director Experience - Total Number of Quoted Boards to Date,Ratios - %Change from Last Period,Total Wealth - Total Wealth,Country,Annual Direct Compensation - Bonus,Ratios - Performance/ Total,Director Experience - Number of Qualifications,Director Experience - Age (Yrs),Company Name,Annual- Equity Linked Options - Estimated Options (unexcercisable),Annual- Equity Linked Options - Vesting Date,Director Experience - Avg. Yrs on Other Quoted Boards,Characteristics of Roles - Individual Role,Annual Direct Compensation - D.C Pension,Characteristics of Roles - Director Network Size,Director Experience - Total Number of Other Boards to Date,Annual- Equity Linked Options - Estimated Options (exercisable),Annual Direct Compensation - Other,Accumulated Wealth - Shares,Characteristics of Roles - Time to Retirement,Director Count Totals - Number of Independent NED with past CFO/FD role,ISIN,Ratios - Equity Linked/ Total,Characteristics of Roles - Time in Company,Sector,Director Experience - Total Number of Quoted Current Boards,Characteristics of Roles - Time in Role,Director Experience - Gender,inputfilestem,region,mgmt_type,workbook_name,fragment_number,sheet_name,Month,Year,Current Year,Current Month
0,2.0,NaN,Current,NaN,SD,3.2,NaN,NaN,NaN,NaN,Robert (Bob) Archie Jennings,NaN,NaN,NaN,NaN,13237186434,NaN,NaN,0.0,NaN,British,FTSE 250,3IN,2.0,9811997164,188.0,0.0,NaN,2.0,NaN,188.0,Jersey,NaN,NaN,2.0,66.0,3I INFRASTRUCTURE PLC (3i Infrastructure Ltd p...,NaN,NaN,6.1,Independent NED,NaN,1617,NaN,NaN,NaN,188.0,4.0,0.0,JE00BF5FX167,NaN,3.2,Investment Companies,2.0,3.2,M,Europe - Board Summary,Europe,Director,Board Summary,NaN,NaN,Mar,2021,2021,Mar
1,0.0,NaN,Current,NaN,SD,1.2,NaN,NaN,NaN,NaN,Samantha (Sam) Jane Hoe-Richardson,NaN,2.0,NaN,NaN,13534906790,NaN,NaN,0.0,NaN,British,FTSE 250,3IN,4.0,9811997164,5.0,0.0,NaN,2.0,NaN,5.0,Jersey,NaN,NaN,2.0,50.0,3I INFRASTRUCTURE PLC (3i Infrastructure Ltd p...,NaN,NaN,8.2,Independent NED,NaN,1486,NaN,NaN,NaN,5.0,19.8,0.0,JE00BF5FX167,NaN,1.2,Investment Companies,2.0,1.2,F,Europe - Board Summary,Europe,Director,Board Summary,NaN,NaN,Mar,2021,2021,Mar
2,1.0,NaN,Current,NaN,SD,8.0,NaN,NaN,NaN,NaN,Paul A Masterton,NaN,3.0,NaN,NaN,13584596849,NaN,NaN,0.0,NaN,NaN,FTSE 250,3IN,5.0,9811997164,100.0,0.0,NaN,1.0,NaN,100.0,Jersey,NaN,NaN,3.0,NaN,3I INFRASTRUCTURE PLC (3i Infrastructure Ltd p...,NaN,NaN,0.0,Senior Independent NED,NaN,1509,NaN,NaN,NaN,100.0,NaN,0.0,JE00BF5FX167,NaN,8.0,Investment Companies,1.0,4.3,M,Europe - Board Summary,Europe,Director,Board Summary,NaN,NaN,Mar,2021,2021,Mar
3,NaN,NaN,Current,NaN,SD,6.9,NaN,NaN,NaN,NaN,Ian Matthew Lobley,NaN,4.0,NaN,NaN,13665216944,NaN,NaN,0.0,NaN,British,FTSE 250,3IN,20.0,9811997164,NaN,0.0,NaN,1.0,NaN,NaN,Jersey,NaN,NaN,1.0,58.0,3I INFRASTRUCTURE PLC (3i Infrastructure Ltd p...,NaN,NaN,0.0,NED,NaN,1129,NaN,NaN,NaN,NaN,11.8,0.0,JE00BF5FX167,NaN,6.9,Investment Companies,1.0,6.9,M,Europe - Board Summary,Europe,Director,Board Summary,NaN,Na

# Additional 1: Get CEO/CFO variables

In [11]:
# Add indicator variables for whether role is CEO or CFO
df['CEO'] = df['Characteristics of Roles - Individual Role'].str.contains(r'^(CEO|CEO/.*)$', case=True, regex=True)
df['CFO'] = df['Characteristics of Roles - Individual Role'].str.contains(r'^(CFO|CFO/.*)$', case=True, regex=True)
# View strings tagged as CEO or CFO
print('CEO roles:', df[df['CEO'] == True]['Characteristics of Roles - Individual Role'].unique())
print('CFO roles:', df[df['CFO'] == True]['Characteristics of Roles - Individual Role'].unique())
# Get number of directors who have been CEOs or CFOs
print('Number of CEOs:', df[df['CEO'] == True]['DirectorID*'].nunique())
print('Number of CFOs:', df[df['CFO'] == True]['DirectorID*'].nunique())

C:\Users\jasonjia\AppData\Local\Temp\ipykernel_10356\1641893263.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df['CEO'] = df['Characteristics of Roles - Individual Role'].str.contains(r'^(CEO|CEO/.*)$', case=True, regex=True)
C:\Users\jasonjia\AppData\Local\Temp\ipykernel_10356\1641893263.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df['CFO'] = df['Characteristics of Roles - Individual Role'].str.contains(r'^(CFO|CFO/.*)$', case=True, regex=True)


CEO roles: ['CEO/MD' 'CEO' 'CEO/MD/General Manager' 'CEO/General Manager' 'CEO/COO'
 'CEO/CFO' 'CEO/Chief Commercial Officer' 'CEO/Acting CFO'
 'CEO/Division MD' 'CEO/Division President' 'CEO/CFO/COO'
 'CEO/Interim CFO' 'CEO/Interim Chief Human Resource Officer'
 'CEO/Interim COO' 'CEO/MD/COO' 'CEO/Interim General Counsel'
 'CEO/Chief Industrial Assets Officer' 'CEO/Interim Head of Division'
 'CEO/Co-COO' 'CEO/CIO' 'CEO/Division Head' 'CEO/Treasurer'
 'CEO/General Counsel/Secretary' 'CEO/Managing Partner'
 'CEO/Chief Investment Officer' 'CEO/Acting Chairman'
 'CEO/Regional President' 'CEO/Co-Chief Investment Officer'
 'CEO/Secretary' 'CEO/Chief Diversity Officer'
 'CEO/Chief Restructuring Officer' 'CEO/CTO' 'CEO/Chief Science Officer'
 'CEO/Acting Regional President' 'CEO/Medical Director'
 'CEO/Director - IR' 'CEO/Deputy CEO'
 'CEO/Interim Chief Business Development & Mktg Officer'
 'CEO/Interim Chief Planning and Business Development Officer'
 'CEO/Interim Director - IR' 'CEO/Acting 

## Check if there is only one CEO and CFO per (company, year)
- We see that there are a few (company, years) with 2 CEOs and/or 2 CFOs. During aggregation, we use 'first' to take the first one.

In [12]:
# Check if there is only one CEO and CFO per (company, year)
ceo_cfo_count_check = df.groupby(['Year', 'CompanyID*'])[['CEO','CFO']].sum().reset_index()
ceo_cfo_count_check

,Year,CompanyID*,CEO,CFO
0,2002,270810,0,0
1,2002,493378,0,0
2,2002,497536,0,1
3,2002,7012097,0,1
4,2002,7168536,0,0
...,...,...,...,...
45167,2022,3584311110,0,0
45168,2022,3675911726,0,1
45169,2022,25422208584,0,1
45170,2022,52870312476,0,0


In [13]:
ceo_cfo_count_check['CEO'].value_counts()

0    34238
1    10903
2       31
Name: CEO, dtype: int64

In [14]:
ceo_cfo_count_check['CFO'].value_counts()

0    29845
1    15257
2       70
Name: CFO, dtype: int64

In [15]:
ceo_cfo_count_check[ceo_cfo_count_check['CFO'] == 2]

,Year,CompanyID*,CEO,CFO
344,2002,197897833,0,2
382,2002,211648958,0,2
622,2002,391122365,0,2
766,2002,434558155,1,2
790,2002,686011336,0,2
1637,2003,211648958,0,2
2015,2003,410745679,0,2
2114,2003,434558155,1,2
3065,2004,183266447,1,2
3570,2004,410745679,0,2


In [16]:
df[(df['CFO']) & (df['Year'] == 2020) & (df['CompanyID*'] == 9723136113)]

,Ratios - Wealth Delta,Total - Total Annual Compensation,Annual Report Year,Annual- Equity Linked Options - Intrinsic Options (excercisable),Director Type,Characteristics of Roles - Time on Board,Accumulated Wealth - LTIPS(max),Ratios - Bonus/ (Bonus&Salary),Annual- Equity Linked Options - Total Equity Linked Compensation,Annual- Equity Linked Options - Share Price,Individual Name,Annual- Equity Linked Options - LTIPS(max),Director Experience - Total Number of Private Current Boards,Annual Direct Compensation - Salary,Director Experience - Total Number of Other Current Boards,DirectorID*,Accumulated Wealth - Intrinsic Option,Accumulated Wealth - Estimated Option,Annual Direct Compensation - Total Salary+Bonus,Annual- Equity Linked Options - Shares,Director Experience - Nationality Mix,Index,Ticker,Director Experience - Total Number of Private Boards to Date,CompanyID*,Accumulated Wealth - Liquid Wealth,Annual Direct Compensation - Total Inc. D.C. Pension & Other,Annual- Equity Linked Options - Intrinsic Options (unexercisable),Director Experience - Total Number of Quoted Boards to Date,Ratios - %Change from Last Period,Total Wealth - Total Wealth,Country,Annual Direct Compensation - Bonus,Ratios - Performance/ Total,Director Experience - Number of Qualifications,Director Experience - Age (Yrs),Company Name,Annual- Equity Linked Options - Estimated Options (unexcercisable),Annual- Equity Linked Options - Vesting Date,Director Experience - Avg. Yrs on Other Quoted Boards,Characteristics of Roles - Individual Role,Annual Direct Compensation - D.C Pension,Characteristics of Roles - Director Network Size,Director Experience - Total Number of Other Boards to Date,Annual- Equity Linked Options - Estimated Options (exercisable),Annual Direct Compensation - Other,Accumulated Wealth - Shares,Characteristics of Roles - Time to Retirement,Director Count Totals - Number of Independent NED with past CFO/FD role,ISIN,Ratios - Equity Linked/ Total,Characteristics of Roles - Time in Company,Sector,Director Experience - Total Number of Quoted Current Boards,Characteristics of Roles - Time in Role,Director Experience - Gender,inputfilestem,region,mgmt_type,workbook_name,fragment_number,sheet_name,Month,Year,Current Year,Current Month,CEO,CFO
1120816,NaN,NaN,Current,NaN,ED,1.0,NaN,NaN,NaN,NaN,Richard Jacques Dumas,NaN,NaN,NaN,NaN,23820456908,NaN,NaN,NaN,NaN,NaN,BOVESPA,EGIE3,NaN,9723136113,NaN,NaN,NaN,1.0,NaN,NaN,Brazil,NaN,NaN,3.0,50.0,ENGIE BRASIL ENERGIA SA (Tractebel Energia SA ...,NaN,NaN,0.0,CFO,NaN,665,NaN,NaN,NaN,NaN,14.8,1.0,BREGIEACNOR9,NaN,5.3,Electricity,1.0,1.2,M,ROW - Board Summary - 1,ROW,Director,Board Summary,1.0,NaN,Dec,2020,2020,Dec,False,True
1120916,NaN,NaN,Current,NaN,SM,NaN,NaN,NaN,NaN,NaN,Marcelo Cardoso Malta,NaN,NaN,NaN,NaN,12002824706,NaN,NaN,NaN,NaN,NaN,BOVESPA,EGIE3,NaN,9723136113,NaN,NaN,NaN,NaN,NaN,NaN,Brazil,NaN,NaN,NaN,NaN,ENGIE BRASIL ENERGIA SA (Tractebel Energia SA ...,NaN,NaN,NaN,CFO,NaN,70,NaN,NaN,NaN,NaN,NaN,NaN,BREGIEACNOR9,NaN,NaN,Electricity,NaN,NaN,NaN,ROW - SMDEs Org Summary - 2,ROW,SMDE,SMDEs Org Summary,2.0,NaN,Dec,2020,2020,Dec,False,True


# Additional 2: Get educational achievements of board members and CEO/CFOs

## We eventually want the following indicator variables:
- Bachelors
- Masters
- MBA
- PhD
- % of all board members with bachelor's
- % of all board members with master's
- % of all board members with MBA
- % of all board members with PhD

For now, we ignore qualification date, so if someone got a Masters in 2020, we flag masters = 1 for all observations, even for those where he/she was an executive before 2020.

In [17]:
# Read in education file
df_education = pd.read_csv('Profile - Education & Achievements.csv', low_memory=False)
df_education = df_education[df_education['sheet_name'] == 'Education']
colstokeep = ['Institution Name', 'InstitutionID*', 'Qualification', 'Country', 'DirectorID*']
df_education = df_education[colstokeep]
# Remove rows with NA values for Qualification
df_education = df_education[~df_education['Qualification'].isna()]
df_education

,Institution Name,InstitutionID*,Qualification,Country,DirectorID*
35760,Universiteit Leiden (University of Leiden),4.670011e+09,Masters Degree,Netherlands,206397712886
35761,SIOO,6.302261e+10,Masters Degree,Netherlands,25356448525
35762,Johns Hopkins University,7.338939e+08,MA,United States,51690811635
35763,University of South Florida (USF),7.519864e+08,BA,United States,51690811635
35764,De Haagse Hogeschool (The Hague University),1.395159e+10,Bachelor's Degree,Netherlands,13726047014
...,...,...,...,...,...
2629545,Geological Society of London (UK),6.102061e+10,Chartered Geologist,United Kingdom - England,5482944327
2629546,Imperial College London (The Imperial College ...,7.567670e+08,Diploma,United Kingdom - England,5482944327
2629547,Imperial College London (The Imperial College ...,7.567670e+08,MSc,United Kingdom - England,5482944327
2629548,Institute of Materials Minerals and Mining (IOM3),7.615274e+08,Fellow,United Kingdom - England,5482944327


## Identify Bachelors, Masters, MBA and PhD degrees

In [18]:
# Add indicator variables for whether degree is Bachelors, Masters, MBA or PhD
df_education['Bachelors'] = df_education['Qualification'].str.contains(r'^(BA.*|BS.*|AB.*|LLB.*|Bachelor.*|.*Graduated.*|Degree.*|B.*)$', case=True, regex=True)
df_education['Masters'] = df_education['Qualification'].str.contains(r'^(MS.*|MSc.*|MPhil.*|MA.*|MEng.*|JD.*|MPA.*|.*MBA.*|.*Master.*Business Administration.*|Master.*)$', case=True, regex=True)
df_education['MBA'] = df_education['Qualification'].str.contains('MBA|Master.*Business Administration', case=True, regex=True)
df_education['PhD'] = df_education['Qualification'].str.contains('PhD', case=True, regex=True)
df_education

C:\Users\jasonjia\AppData\Local\Temp\ipykernel_10356\1044042464.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_education['Bachelors'] = df_education['Qualification'].str.contains(r'^(BA.*|BS.*|AB.*|LLB.*|Bachelor.*|.*Graduated.*|Degree.*|B.*)$', case=True, regex=True)
C:\Users\jasonjia\AppData\Local\Temp\ipykernel_10356\1044042464.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_education['Masters'] = df_education['Qualification'].str.contains(r'^(MS.*|MSc.*|MPhil.*|MA.*|MEng.*|JD.*|MPA.*|.*MBA.*|.*Master.*Business Administration.*|Master.*)$', case=True, regex=True)


,Institution Name,InstitutionID*,Qualification,Country,DirectorID*,Bachelors,Masters,MBA,PhD
35760,Universiteit Leiden (University of Leiden),4.670011e+09,Masters Degree,Netherlands,206397712886,False,True,False,False
35761,SIOO,6.302261e+10,Masters Degree,Netherlands,25356448525,False,True,False,False
35762,Johns Hopkins University,7.338939e+08,MA,United States,51690811635,False,True,False,False
35763,University of South Florida (USF),7.519864e+08,BA,United States,51690811635,True,False,False,False
35764,De Haagse Hogeschool (The Hague University),1.395159e+10,Bachelor's Degree,Netherlands,13726047014,True,False,False,False
...,...,...,...,...,...,...,...,...,...
2629545,Geological Society of London (UK),6.102061e+10,Chartered Geologist,United Kingdom - England,5482944327,False,False,False,False
2629546,Imperial College London (The Imperial College ...,7.567670e+08,Diploma,United Kingdom - England,5482944327,False,False,False,False
2629547,Imperial College London (The Imperial College ...,7.567670e+08,MSc,United Kingdom - England,5482944327,False,True,False,False
2629548,Institute of Materials Minerals and Mining (IOM3),7.615274e+08,Fellow,United Kingdom - England,5482944327,False,False,False,False


### Check regex to see that it's working as intended

In [19]:
df_education_check = df_education[['Qualification', 'MBA', 'PhD', 'Bachelors', 'Masters']]
# MBA is a Masters' program. But a Bachelor's is not a Masters, etc. So we look at the sum of booleans for each row to check if our regex patterns are correct.
df_education_check['sum'] = df_education_check[['MBA', 'PhD', 'Bachelors', 'Masters']].sum(axis=1)
df_education_check

C:\Users\jasonjia\AppData\Local\Temp\ipykernel_10356\366615561.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_education_check['sum'] = df_education_check[['MBA', 'PhD', 'Bachelors', 'Masters']].sum(axis=1)


,Qualification,MBA,PhD,Bachelors,Masters,sum
35760,Masters Degree,False,False,False,True,1
35761,Masters Degree,False,False,False,True,1
35762,MA,False,False,False,True,1
35763,BA,False,False,True,False,1
35764,Bachelor's Degree,False,False,True,False,1
...,...,...,...,...,...,...
2629545,Chartered Geologist,False,False,False,False,0
2629546,Diploma,False,False,False,False,0
2629547,MSc,False,False,False,True,1
2629548,Fellow,False,False,False,False,0


In [20]:
# Check those not classified as any degree
df_education_check[df_education_check['sum'] == 0]['Qualification'].unique()[:100]

array(['Executive Education Program', 'Advanced Management Program',
       'LLM', 'CPA', 'Registered Accountant (RA)', 'Studied',
       'Chartered Accountant', 'Completed',
       'International Baccalaureate (IB)', 'Executive Leadership Program',
       'MEc', "CEMS Master's in International Management (CEMS MIM)",
       'Attended', 'Diploma', 'Chartered Financial Analyst',
       'Certified Public Accountant', 'Electronical Engineering',
       'Senior Management Program', 'Management Development Program',
       'Associate', 'Certified', 'AMP', 'CFA', 'Dipl.-kfm',
       'Executive Management Program', 'Postgraduate Studies',
       'General Management Program', 'Leadership Program',
       'Executive Program', 'Training Program', 'Doctor of Law',
       'Postgraduate Degree', 'CA', 'Dipl Kfm',
       "DESS (Diplôme d'études supérieures spécialisées)", 'Dipl Ing',
       'Doctorate', 'Senior Executive Program', 'Chartered Engineer',
       'Fellow', 'Lic oec', 'Graduate Diploma',

In [21]:
# Check that no qualification belongs to more than 2 categories
df_education_check['sum'].value_counts()

1    1139019
0     482259
2     196483
Name: sum, dtype: int64

In [22]:
# Check classifications
print("Not classified as any degree:", df_education_check[df_education_check['sum'] == 0]['Qualification'].unique())
print("Bachelors:", df_education_check[df_education_check['Bachelors'] == True]['Qualification'].unique())
print("Masters:", df_education_check[df_education_check['Masters'] == True]['Qualification'].unique())
print("MBA:", df_education_check[df_education_check['MBA'] == True]['Qualification'].unique())
print("PhD:", df_education_check[df_education_check['PhD'] == True]['Qualification'].unique())

Not classified as any degree: ['Executive Education Program' 'Advanced Management Program' 'LLM' ...
 'Certificate in International Treasury Management (CertITM)'
 'Certificate in Relationship Management (CertRM)'
 'Modern European Studies']
Bachelors: ['BA' "Bachelor's Degree" 'Graduated' 'BSc' 'Degree'
 'Bachelor of Administration (B.Admin) (Hons)' 'AB' 'LLB' 'BSBA'
 'BSc (Hons)' 'BBA' 'BComm' 'BA (Hons)' 'BCom' 'BE' 'BEng' 'BS'
 'BA (magna cum laude)' 'Graduated (magna cum laude)' 'BSc (Distinction)'
 'BTech' 'BS (summa Cum Laude)' 'Graduated (Hons)' 'Baccalaureate'
 "Bachelor's Degree (magna cum laude)" 'BS (Hons)' 'BCom (Hons)' 'BSEE'
 'BS (magna Cum Laude)' 'BSc (magna cum laude)' 'BA (Cum Laude)'
 'LLB (Hons)' 'Degree (Hons)' 'BBA (summa cum laude)'
 'Graduated (cum laude)' 'Degree (summa cum laude)' 'BEng (Hons)'
 'BComm (Hons)' 'BA (summa cum laude)' 'BTech (Hons)'
 'BBS (Bachelor of Business Studies)' 'BEd'
 "Bachelor's Degree (cum laude)" 'BSc (summa cum laude)'
 "Bachelor's

In [23]:
df_education['Qualification'].value_counts().iloc[:100]

MBA                                177582
BA                                 147811
BS                                 144399
Graduated                           98197
JD                                  72975
Bachelor's Degree                   69589
Masters Degree                      60462
PhD                                 54337
Studied                             47300
Degree                              43932
MS                                  42928
Certified Public Accountant         39082
BSc                                 39050
Certified                           35313
MSc                                 32574
MA                                  31037
Chartered Accountant                27658
Attended                            26705
BBA                                 26329
Diploma                             24464
BA (Hons)                           22767
LLB                                 22452
LLM                                 17338
BCom                              

## Get Bachelors, Masters, MBA and PhD 'indicator' variables, defined by whether the director has at least one of the corresponding degrees

In [24]:
# For each director, get whether he/she has at least one Bachelors, Masters, MBA and PhD degree.
df_education_has_degree = df_education.groupby('DirectorID*')[['Bachelors', 'Masters', 'MBA', 'PhD']].sum()
df_education_has_degree = df_education_has_degree.applymap(lambda x: 0 if x == 0 else 1)
df_education_has_degree = df_education_has_degree.reset_index()
df_education_has_degree

,DirectorID*,Bachelors,Masters,MBA,PhD
0,1680,1,0,0,0
1,3040,0,0,0,0
2,4890,1,0,0,0
3,6000,1,1,0,0
4,14010,1,1,1,0
...,...,...,...,...,...
742574,207075512928,1,0,0,0
742575,207075612928,1,0,0,0
742576,207076212928,0,0,0,0
742577,207076712928,1,0,0,0


## Get the highest ranked degree for each director

In [25]:
# Assign rank to degrees, so that we can select the highest ranked degree later
def get_rank(row):
    if row['PhD'] == True:
        return 4
    elif row['MBA'] == True:
        return 3
    elif row['Masters'] == True:
        return 2
    elif row['Bachelors'] == True:
        return 1
    else:
        return 0

In [26]:
# For CEO/CFO, we choose the highest ranked degree in the order of: PhD > MBA > Masters > Bachelors > Nothing
df_education['rank'] = df_education.apply(lambda row: get_rank(row), axis=1)
df_education

,Institution Name,InstitutionID*,Qualification,Country,DirectorID*,Bachelors,Masters,MBA,PhD,rank
35760,Universiteit Leiden (University of Leiden),4.670011e+09,Masters Degree,Netherlands,206397712886,False,True,False,False,2
35761,SIOO,6.302261e+10,Masters Degree,Netherlands,25356448525,False,True,False,False,2
35762,Johns Hopkins University,7.338939e+08,MA,United States,51690811635,False,True,False,False,2
35763,University of South Florida (USF),7.519864e+08,BA,United States,51690811635,True,False,False,False,1
35764,De Haagse Hogeschool (The Hague University),1.395159e+10,Bachelor's Degree,Netherlands,13726047014,True,False,False,False,1
...,...,...,...,...,...,...,...,...,...,...
2629545,Geological Society of London (UK),6.102061e+10,Chartered Geologist,United Kingdom - England,5482944327,False,False,False,False,0
2629546,Imperial College London (The Imperial College ...,7.567670e+08,Diploma,United Kingdom - England,5482944327,False,False,False,False,0
2629547,Imperial College London (The Imperial College ...,7.567670e+08,MSc,United Kingdom - England,5482944327,False,True,False,False,2
2629548,Institute of Materials Minerals and Mining (IOM3),7.615274e+08,Fellow,United Kingdom - England,5482944327,False,False,False,False,0


In [27]:
# Get the highest degree rank for each director
df_education_rank = df_education.groupby('DirectorID*')['rank'].max().reset_index()
df_education_rank

,DirectorID*,rank
0,1680,1
1,3040,0
2,4890,1
3,6000,2
4,14010,3
...,...,...
742574,207075512928,1
742575,207075612928,1
742576,207076212928,0
742577,207076712928,1


In [28]:
# Get the first degree of that rank for each director (because some directors have e.g. multiple Masters degrees)
df_education_highest_degree = df_education.merge(df_education_rank, on=['DirectorID*', 'rank'], how='inner').sort_values('DirectorID*')
df_education_highest_degree

,Institution Name,InstitutionID*,Qualification,Country,DirectorID*,Bachelors,Masters,MBA,PhD,rank
380668,Unknown Organisation,3.780000e+02,LLB,Unknown,1680,True,False,False,False,1
387503,Institute of Chartered Accountants of Scotland...,7.812792e+08,Fellow,United Kingdom - Scotland,3040,False,False,False,False,0
39991,Università degli Studi di Roma La Sapienza (La...,7.523565e+08,Degree,Italy,4890,True,False,False,False,1
30279,Sciences Po (Institut d'Etudes Politiques de P...,7.564669e+08,Masters Degree,France,6000,False,True,False,False,2
100007,University of Chicago Booth School of Business...,1.400243e+10,MBA,United States,14010,False,True,True,False,3
...,...,...,...,...,...,...,...,...,...,...
605033,Ben-Gurion University of the Negev,7.242715e+08,BSc,Israel,207075512928,True,False,False,False,1
908212,University of Cape Town (UCT),5.299564e+08,BCom,South Africa,207075612928,True,False,False,False,1
321572,University of Western Australia,7.533466e+08,Management Education Programme,Australia,207076212928,False,False,False,False,0
867444,University of Salford,7.517064e+08,Degree (Hons),United Kingdom - England,207076712928,True,False,False,False,1


In [29]:
df_education_highest_degree = df_education_highest_degree.groupby('DirectorID*').first().reset_index()
df_education_highest_degree

,DirectorID*,Institution Name,InstitutionID*,Qualification,Country,Bachelors,Masters,MBA,PhD,rank
0,1680,Unknown Organisation,3.780000e+02,LLB,Unknown,True,False,False,False,1
1,3040,Institute of Chartered Accountants of Scotland...,7.812792e+08,Fellow,United Kingdom - Scotland,False,False,False,False,0
2,4890,Università degli Studi di Roma La Sapienza (La...,7.523565e+08,Degree,Italy,True,False,False,False,1
3,6000,Sciences Po (Institut d'Etudes Politiques de P...,7.564669e+08,Masters Degree,France,False,True,False,False,2
4,14010,University of Chicago Booth School of Business...,1.400243e+10,MBA,United States,False,True,True,False,3
...,...,...,...,...,...,...,...,...,...,...
742574,207075512928,Ben-Gurion University of the Negev,7.242715e+08,BSc,Israel,True,False,False,False,1
742575,207075612928,University of Cape Town (UCT),5.299564e+08,BCom,South Africa,True,False,False,False,1
742576,207076212928,University of Western Australia,7.533466e+08,Management Education Programme,Australia,False,False,False,False,0
742577,207076712928,University of Salford,7.517064e+08,Degree (Hons),United Kingdom - England,True,False,False,False,1


In [30]:
# Keep desired variables and rename them
df_education_highest_degree = df_education_highest_degree[['DirectorID*', 'Institution Name', 'InstitutionID*', 'Qualification', 'Country']]
df_education_highest_degree.columns = ['DirectorID*', 'Highest Ranked Degree - Institution Name', 'Highest Ranked Degree - InstitutionID*', 'Highest Ranked Degree - Qualification', 'Highest Ranked Degree - Country']
df_education_highest_degree

,DirectorID*,Highest Ranked Degree - Institution Name,Highest Ranked Degree - InstitutionID*,Highest Ranked Degree - Qualification,Highest Ranked Degree - Country
0,1680,Unknown Organisation,3.780000e+02,LLB,Unknown
1,3040,Institute of Chartered Accountants of Scotland...,7.812792e+08,Fellow,United Kingdom - Scotland
2,4890,Università degli Studi di Roma La Sapienza (La...,7.523565e+08,Degree,Italy
3,6000,Sciences Po (Institut d'Etudes Politiques de P...,7.564669e+08,Masters Degree,France
4,14010,University of Chicago Booth School of Business...,1.400243e+10,MBA,United States
...,...,...,...,...,...
742574,207075512928,Ben-Gurion University of the Negev,7.242715e+08,BSc,Israel
742575,207075612928,University of Cape Town (UCT),5.299564e+08,BCom,South Africa
742576,207076212928,University of Western Australia,7.533466e+08,Management Education Programme,Australia
742577,207076712928,University of Salford,7.517064e+08,Degree (Hons),United Kingdom - England


## Merge additional education variables with df

In [31]:
# Merge with df
df_education_groupby = df_education_has_degree.merge(df_education_highest_degree, on='DirectorID*', how='inner')
df_education_groupby

,DirectorID*,Bachelors,Masters,MBA,PhD,Highest Ranked Degree - Institution Name,Highest Ranked Degree - InstitutionID*,Highest Ranked Degree - Qualification,Highest Ranked Degree - Country
0,1680,1,0,0,0,Unknown Organisation,3.780000e+02,LLB,Unknown
1,3040,0,0,0,0,Institute of Chartered Accountants of Scotland...,7.812792e+08,Fellow,United Kingdom - Scotland
2,4890,1,0,0,0,Università degli Studi di Roma La Sapienza (La...,7.523565e+08,Degree,Italy
3,6000,1,1,0,0,Sciences Po (Institut d'Etudes Politiques de P...,7.564669e+08,Masters Degree,France
4,14010,1,1,1,0,University of Chicago Booth School of Business...,1.400243e+10,MBA,United States
...,...,...,...,...,...,...,...,...,...
742574,207075512928,1,0,0,0,Ben-Gurion University of the Negev,7.242715e+08,BSc,Israel
742575,207075612928,1,0,0,0,University of Cape Town (UCT),5.299564e+08,BCom,South Africa
742576,207076212928,0,0,0,0,University of Western Australia,7.533466e+08,Management Education Programme,Australia
742577,207076712928,1,0,0,0,University of Salford,7.517064e+08,Degree (Hons),United Kingdom - England


In [32]:
df = df.merge(df_education_groupby, on='DirectorID*')
df

,Ratios - Wealth Delta,Total - Total Annual Compensation,Annual Report Year,Annual- Equity Linked Options - Intrinsic Options (excercisable),Director Type,Characteristics of Roles - Time on Board,Accumulated Wealth - LTIPS(max),Ratios - Bonus/ (Bonus&Salary),Annual- Equity Linked Options - Total Equity Linked Compensation,Annual- Equity Linked Options - Share Price,Individual Name,Annual- Equity Linked Options - LTIPS(max),Director Experience - Total Number of Private Current Boards,Annual Direct Compensation - Salary,Director Experience - Total Number of Other Current Boards,DirectorID*,Accumulated Wealth - Intrinsic Option,Accumulated Wealth - Estimated Option,Annual Direct Compensation - Total Salary+Bonus,Annual- Equity Linked Options - Shares,Director Experience - Nationality Mix,Index,Ticker,Director Experience - Total Number of Private Boards to Date,CompanyID*,Accumulated Wealth - Liquid Wealth,Annual Direct Compensation - Total Inc. D.C. Pension & Other,Annual- Equity Linked Options - Intrinsic Options (unexercisable),Director Experience - Total Number of Quoted Boards to Date,Ratios - %Change from Last Period,Total Wealth - Total Wealth,Country,Annual Direct Compensation - Bonus,Ratios - Performance/ Total,Director Experience - Number of Qualifications,Director Experience - Age (Yrs),Company Name,Annual- Equity Linked Options - Estimated Options (unexcercisable),Annual- Equity Linked Options - Vesting Date,Director Experience - Avg. Yrs on Other Quoted Boards,Characteristics of Roles - Individual Role,Annual Direct Compensation - D.C Pension,Characteristics of Roles - Director Network Size,Director Experience - Total Number of Other Boards to Date,Annual- Equity Linked Options - Estimated Options (exercisable),Annual Direct Compensation - Other,Accumulated Wealth - Shares,Characteristics of Roles - Time to Retirement,Director Count Totals - Number of Independent NED with past CFO/FD role,ISIN,Ratios - Equity Linked/ Total,Characteristics of Roles - Time in Company,Sector,Director Experience - Total Number of Quoted Current Boards,Characteristics of Roles - Time in Role,Director Experience - Gender,inputfilestem,region,mgmt_type,workbook_name,fragment_number,sheet_name,Month,Year,Current Year,Current Month,CEO,CFO,Bachelors,Masters,MBA,PhD,Highest Ranked Degree - Institution Name,Highest Ranked Degree - InstitutionID*,Highest Ranked Degree - Qualification,Highest Ranked Degree - Country
0,2.0,NaN,Current,NaN,SD,3.2,NaN,NaN,NaN,NaN,Robert (Bob) Archie Jennings,NaN,NaN,NaN,NaN,13237186434,NaN,NaN,0.0,NaN,British,FTSE 250,3IN,2.0,9811997164,188.0,0.0,NaN,2.0,NaN,188.0,Jersey,NaN,NaN,2.0,66.0,3I INFRASTRUCTURE PLC (3i Infrastructure Ltd p...,NaN,NaN,6.1,Independent NED,NaN,1617,NaN,NaN,NaN,188.0,4.0,0.0,JE00BF5FX167,NaN,3.2,Investment Companies,2.0,3.2,M,Europe - Board Summary,Europe,Director,Board Summary,NaN,NaN,Mar,2021,2021,Mar,False,False,1,0,0,0,University of Bristol (Formerly known as Unive...,7.767289e+08,Graduated,United Kingdom - England
1,1.0,10.0,Mar 2018,NaN,SD,0.1,NaN,NaN,NaN,NaN,Robert (Bob) Archie Jennings,NaN,1.0,10.0,NaN,13237186434,NaN,NaN,10.0,NaN,British,FTSE 250,3IN,2.0,9811997164,122.0,10.0,NaN,2.0,NaN,122.0,Jersey,NaN,NaN,2.0,62.0,3I INFRASTRUCTURE PLC (3i Infrastructure Ltd p...,NaN,NaN,3.1,Independent NED,NaN,1617,NaN,NaN,NaN,122.0,7.1,1.0,JE00BF5FX167,NaN,0.1,Investment Companies,2.0,0.1,M,Europe - Board Summary,Europe,Director,Board Summary,NaN,NaN,Mar,2018,2021,Mar,False,False,1,0,0,0,University of Bristol (Formerly known as Unive...,7.767289e+08,Graduated,United Kingdom - England
2,2.0,63.0,Mar 2019,NaN,SD,1.1,NaN,NaN,NaN,NaN,Robert (Bob) Archie Jennings,NaN,1.0,63.0,NaN,13237186434,NaN,NaN,63.0,NaN,British,FTSE 250,3IN,2.0,9811997164,192.0,63.0,NaN,2.0,500.00,192.0,Jersey,NaN,NaN,2.0,63.0,3I INFRASTRUCTURE PLC (3i Infrastructure Ltd p...,NaN,NaN,4.1,Independent NED,NaN,1617,NaN,NaN,NaN,192.0,6.1,1.0,JE00BF5FX167,NaN,1.1,Investment Companies,2.0,1.1,M,Europe - Board Summary,Europe,Director,Bo

# Base: Groupby year and company for applicable variables to get base manager-level dataset

In [33]:
# View columns
df.columns

Index(['Ratios - Wealth Delta', 'Total - Total Annual Compensation',
       'Annual Report Year',
       'Annual- Equity Linked Options - Intrinsic Options (excercisable)',
       'Director Type', 'Characteristics of Roles - Time on Board',
       'Accumulated Wealth - LTIPS(max)', 'Ratios - Bonus/ (Bonus&Salary)',
       'Annual- Equity Linked Options - Total Equity Linked Compensation',
       'Annual- Equity Linked Options - Share Price', 'Individual Name',
       'Annual- Equity Linked Options - LTIPS(max)',
       'Director Experience - Total Number of Private Current Boards',
       'Annual Direct Compensation - Salary',
       'Director Experience - Total Number of Other Current Boards',
       'DirectorID*', 'Accumulated Wealth - Intrinsic Option',
       'Accumulated Wealth - Estimated Option',
       'Annual Direct Compensation - Total Salary+Bonus',
       'Annual- Equity Linked Options - Shares',
       'Director Experience - Nationality Mix', 'Index', 'Ticker',
       'Dir

In [34]:
def create_agg_dict(agg_list, function):
    agg_dict = {}
    for var in agg_list:
        agg_dict[var] = function
    return agg_dict

In [35]:
agg_list_first = ['Country', 'ISIN', 'region']
agg_list_mean = ['Characteristics of Roles - Director Network Size', 'Characteristics of Roles - Time to Retirement', 
                 'Characteristics of Roles - Time in Role', 'Characteristics of Roles - Time on Board', 
                 'Characteristics of Roles - Time in Company',
                 'Director Experience - Total Number of Quoted Boards to Date', 'Director Experience - Total Number of Private Boards to Date', 
                 'Director Experience - Total Number of Other Boards to Date', 
                 'Director Experience - Total Number of Quoted Current Boards', 'Director Experience - Total Number of Private Current Boards', 
                 'Director Experience - Total Number of Other Current Boards', 
                 'Director Experience - Avg. Yrs on Other Quoted Boards', 'Director Experience - Age (Yrs)', 
                 'Director Experience - Number of Qualifications',
                 'Ratios - Bonus/ (Bonus&Salary)','Ratios - Equity Linked/ Total', 
                 'Ratios - Performance/ Total', 'Ratios - %Change from Last Period', 'Ratios - Wealth Delta',
                 'Director Count Totals - Number of Independent NED with past CFO/FD role',
                 'Annual Direct Compensation - Salary',  'Annual Direct Compensation - Bonus', 
                 'Annual Direct Compensation - D.C Pension', 'Annual Direct Compensation - Other', 
                 'Annual Direct Compensation - Total Salary+Bonus', 
                 'Annual Direct Compensation - Total Inc. D.C. Pension & Other',
                 'Annual- Equity Linked Options - Shares', 'Annual- Equity Linked Options - LTIPS(max)',
                 'Annual- Equity Linked Options - Intrinsic Options (excercisable)',  
                 'Annual- Equity Linked Options - Intrinsic Options (unexercisable)',
                 'Annual- Equity Linked Options - Estimated Options (unexcercisable)',
                 'Annual- Equity Linked Options - Estimated Options (exercisable)',
                 'Annual- Equity Linked Options - Share Price', 'Annual- Equity Linked Options - Total Equity Linked Compensation', 
                 'Total - Total Annual Compensation',
                 'Accumulated Wealth - Shares', 'Accumulated Wealth - LTIPS(max)', 
                 'Accumulated Wealth - Intrinsic Option', 'Accumulated Wealth - Estimated Option', 
                 'Accumulated Wealth - Liquid Wealth', 
                 'Total Wealth - Total Wealth',
                 'Bachelors', 'Masters',
                 'MBA', 'PhD']
agg_list_list = ['Director Experience - Nationality Mix']

agg_dict_first = create_agg_dict(agg_list_first, 'first')
agg_dict_mean = create_agg_dict(agg_list_mean, 'mean')
# agg_dict_list = create_agg_dict(agg_list_list, list)

agg_dict = {**agg_dict_first, **agg_dict_mean}
# agg_dict = {**agg_dict_first, **agg_dict_mean, **agg_dict_list}
agg_dict = OrderedDict(agg_dict)
agg_dict

OrderedDict([('Country', 'first'),
             ('ISIN', 'first'),
             ('region', 'first'),
             ('Characteristics of Roles - Director Network Size', 'mean'),
             ('Characteristics of Roles - Time to Retirement', 'mean'),
             ('Characteristics of Roles - Time in Role', 'mean'),
             ('Characteristics of Roles - Time on Board', 'mean'),
             ('Characteristics of Roles - Time in Company', 'mean'),
             ('Director Experience - Total Number of Quoted Boards to Date',
              'mean'),
             ('Director Experience - Total Number of Private Boards to Date',
              'mean'),
             ('Director Experience - Total Number of Other Boards to Date',
              'mean'),
             ('Director Experience - Total Number of Quoted Current Boards',
              'mean'),
             ('Director Experience - Total Number of Private Current Boards',
              'mean'),
             ('Director Experience - Total Number

In [36]:
# See how many year-months a company has
print(df.groupby(['CompanyID*', 'Company Name', 'Sector'])['Annual Report Year'].nunique().max())
print(df.groupby(['CompanyID*', 'Company Name', 'Sector'])['Year'].nunique().max())
# This means that each company only has 1 year-month a year.
# 2022 - 2002 + 1 = 21

21
21


In [37]:
# Note: CompanyID and CompanyID + Company Name both gave same number of rows
agg_dict_final = dict(list(agg_dict.items())[:]) # subset for debugging
# We aggregate to the year level
df_groupby = df.groupby(['Year', 'CompanyID*', 'Company Name', 'Sector']).agg(agg_dict_final).reset_index()
df_groupby

,Year,CompanyID*,Company Name,Sector,Country,ISIN,region,Characteristics of Roles - Director Network Size,Characteristics of Roles - Time to Retirement,Characteristics of Roles - Time in Role,Characteristics of Roles - Time on Board,Characteristics of Roles - Time in Company,Director Experience - Total Number of Quoted Boards to Date,Director Experience - Total Number of Private Boards to Date,Director Experience - Total Number of Other Boards to Date,Director Experience - Total Number of Quoted Current Boards,Director Experience - Total Number of Private Current Boards,Director Experience - Total Number of Other Current Boards,Director Experience - Avg. Yrs on Other Quoted Boards,Director Experience - Age (Yrs),Director Experience - Number of Qualifications,Ratios - Bonus/ (Bonus&Salary),Ratios - Equity Linked/ Total,Ratios - Performance/ Total,Ratios - %Change from Last Period,Ratios - Wealth Delta,Director Count Totals - Number of Independent NED with past CFO/FD role,Annual Direct Compensation - Salary,Annual Direct Compensation - Bonus,Annual Direct Compensation - D.C Pension,Annual Direct Compensation - Other,Annual Direct Compensation - Total Salary+Bonus,Annual Direct Compensation - Total Inc. D.C. Pension & Other,Annual- Equity Linked Options - Shares,Annual- Equity Linked Options - LTIPS(max),Annual- Equity Linked Options - Intrinsic Options (excercisable),Annual- Equity Linked Options - Intrinsic Options (unexercisable),Annual- Equity Linked Options - Estimated Options (unexcercisable),Annual- Equity Linked Options - Estimated Options (exercisable),Annual- Equity Linked Options - Share Price,Annual- Equity Linked Options - Total Equity Linked Compensation,Total - Total Annual Compensation,Accumulated Wealth - Shares,Accumulated Wealth - LTIPS(max),Accumulated Wealth - Intrinsic Option,Accumulated Wealth - Estimated Option,Accumulated Wealth - Liquid Wealth,Total Wealth - Total Wealth,Bachelors,Masters,MBA,PhD
0,2002,270810,MDU RESOURCES GROUP INC,Utilities - Other,United States,US5526901096,None,654.333333,7.681818,8.225000,10.500000,12.483333,1.600000,4.777778,1.0,1.400000,1.714286,1.0,3.090000,60.363636,2.416667,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.466667,0.200000,0.066667
1,2002,493378,FIRST COMMONWEALTH FINANCIAL CORP PA,Banks,United States,US3198291078,None,406.785714,6.300000,4.766667,9.712500,9.511111,1.125000,8.222222,NaN,1.000000,3.333333,NaN,0.000000,61.000000,1.777778,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.357143,0.214286,0.000000
2,2002,497536,FIRST COMMUNITY BANKSHARES INC (First Communit...,Banks,United States,US31983A1034,None,513.200000,10.700000,7.760000,11.185714,9.800000,1.000000,5.111111,1.0,1.000000,1.777778,1.0,0.000000,56.800000,2.400000,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.900000,0.300000,0.000000,0.100000
3,2002,7012097,FIRST INDUSTRIAL REALTY TRUST INC,Real Estate,United States,US32054K1034,None,1720.944444,14.046154,6.061538,7.766667,7.769231,3.000000,4.444444,NaN,1.666667,1.500000,NaN,3.577778,53.076923,1.923077,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.944444,0.555556,0.444444,0.000000
4,2002,7168536,BENCHMARK ELECTRONICS INC,Electronic & Electrical Equipment,United States,US08160H1014,None,605.857143,13.166667,5.483333,10.666667,13.816667,1.200000,2.500000,NaN,1.000000,NaN,NaN,1.833333,53.000000,2.333333,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.857143,0.714286,0.285714,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45162,2022,3584311110,SLEEP NUMBER C

# Additional 3: Get shares by gender and nationality

In [38]:
# Get shares by gender
df_charac = pd.DataFrame(df.groupby(['CompanyID*', 'Year', 'Director Experience - Gender'])['Director Experience - Gender'].count())
df_charac.columns = ['count']
df_charac = df_charac.reset_index()
df_charac = df_charac.pivot(index=['CompanyID*', 'Year'], columns='Director Experience - Gender', values='count')
df_charac = df_charac.fillna(0)
df_charac['sum'] = df_charac.sum(axis=1)
df_charac['Director Experience - Gender - Share of Males'] = df_charac['M'] / df_charac['sum']
df_charac['Director Experience - Gender - Share of Females'] = df_charac['F'] / df_charac['sum']
df_charac = df_charac.drop(columns=['F', 'M', 'sum'])
df_charac = df_charac.reset_index()
df_charac_gender = df_charac.copy()
df_charac_gender

Director Experience - Gender,CompanyID*,Year,Director Experience - Gender - Share of Males,Director Experience - Gender - Share of Females
0,270810,2002,0.916667,0.083333
1,270810,2003,0.846154,0.153846
2,270810,2004,0.846154,0.153846
3,270810,2005,0.769231,0.230769
4,270810,2006,0.750000,0.250000
...,...,...,...,...
44494,306519112781,2018,0.777778,0.222222
44495,306519112781,2019,0.777778,0.222222
44496,306519112781,2020,0.800000,0.200000
44497,307364612844,2019,0.875000,0.125000


In [39]:
# Get counts by nationality
df_charac = pd.DataFrame(df.groupby(['CompanyID*', 'Year', 'Director Experience - Nationality Mix'])['Director Experience - Nationality Mix'].count())
df_charac.columns = ['count']
df_charac = df_charac.reset_index()
df_charac = df_charac.pivot(index=['CompanyID*', 'Year'], columns='Director Experience - Nationality Mix', values='count')
df_charac = df_charac.fillna(0)

# Get most common nationalities, but return NaN if there is a tie
most_common_nationalities_var_list = ['Director Experience - Nationality Mix - Most Common Nationality']
# , 'Director Experience - Nationality Mix - 2nd Most Common Nationality', 'Director Experience - Nationality Mix - 3rd Most Common Nationality']
df_charac[most_common_nationalities_var_list] = df_charac.apply(lambda row: pd.Series(row.sort_values(ascending=False).index[:1]), axis=1)
# df_charac[most_common_nationalities_var_list] = df_charac.apply(lambda row: pd.Series(row.sort_values(ascending=False).index[:3]), axis=1)

# Get shares by nationality
df_charac['sum'] = df_charac.drop(columns=most_common_nationalities_var_list).sum(axis=1)
for charac in df_charac.columns.drop(['sum'] + most_common_nationalities_var_list):
    df_charac[f'Director Experience - Nationality Mix - Share of {charac}'] = df_charac[charac] / df_charac['sum']
    df_charac = df_charac.drop(columns=[charac])
df_charac = df_charac.drop(columns='sum')

df_charac = df_charac.reset_index()
df_charac_nationality = df_charac.copy()
df_charac_nationality

Director Experience - Nationality Mix,CompanyID*,Year,Director Experience - Nationality Mix - Most Common Nationality,Director Experience - Nationality Mix - Share of Algerian,Director Experience - Nationality Mix - Share of American,Director Experience - Nationality Mix - Share of Angolan,Director Experience - Nationality Mix - Share of Argentine,Director Experience - Nationality Mix - Share of Armenian,Director Experience - Nationality Mix - Share of Australian,Director Experience - Nationality Mix - Share of Austrian,Director Experience - Nationality Mix - Share of Bahraini,Director Experience - Nationality Mix - Share of Barbadian,Director Experience - Nationality Mix - Share of Belarusian,Director Experience - Nationality Mix - Share of Belgian,Director Experience - Nationality Mix - Share of Bermudian,Director Experience - Nationality Mix - Share of Bolivian,Director Experience - Nationality Mix - Share of Bosnian,Director Experience - Nationality Mix - Share of Brazilian,Director Experience - Nationality Mix - Share of British,Director Experience - Nationality Mix - Share of Bulgarian,Director Experience - Nationality Mix - Share of Burkinabe,Director Experience - Nationality Mix - Share of Burmese,Director Experience - Nationality Mix - Share of Canadian,Director Experience - Nationality Mix - Share of Caymanian,Director Experience - Nationality Mix - Share of Chilean,Director Experience - Nationality Mix - Share of Chinese,Director Experience - Nationality Mix - Share of Chinese (Taiwan),Director Experience - Nationality Mix - Share of Colombian,Director Experience - Nationality Mix - Share of Congolese (DR Congo),Director Experience - Nationality Mix - Share of Costa Rican,Director Experience - Nationality Mix - Share of Croatian,Director Experience - Nationality Mix - Share of Cuban,Director Experience - Nationality Mix - Share of Cypriot,Director Experience - Nationality Mix - Share of Czech,Director Experience - Nationality Mix - Share of Danish,Director Experience - Nationality Mix - Share of Dutch,Director Experience - Nationality Mix - Share of Ecuadorian,Director Experience - Nationality Mix - Share of Egyptian,Director Experience - Nationality Mix - Share of Emirian,Director Experience - Nationality Mix - Share of Ethiopian,...,Director Experience - Nationality Mix - Share of Mauritanian,Director Experience - Nationality Mix - Share of Mexican,Director Experience - Nationality Mix - Share of Monacan,Director Experience - Nationality Mix - Share of Moroccan,Director Experience - Nationality Mix - Share of Mozambican,Director Experience - Nationality Mix - Share of New Zealander,Director Experience - Nationality Mix - Share of Nicaraguan,Director Experience - Nationality Mix - Share of Nigerian,Director Experience - Nationality Mix - Share of Norwegian,Director Experience - Nationality Mix - Share of Omani,Director Experience - Nationality Mix - Share of Pakistani,Director Experience - Nationality Mix - Share of Panamanian,Director Experience - Nationality Mix - Share of Peruvian,Director Experience - Nationality Mix - Share of Polish,Director Experience - Nationality Mix - Share of Portuguese,Director Experience - Nationality Mix - Share of Puerto Rican,Director Experience - Nationality Mix - Share of Qatari,Director Experience - Nationality Mix - Share of Romanian,Director Experience - Nationality Mix - Share of Russian,Director Experience - Nationality Mix - Share of Salvadoran,Director Experience - Nationality Mix - Share of Saudi,Director Experience - Nationality Mix - Share of Singaporean,Director Experience - Nationality Mix - Share of Slovak,Director Experience - Nationality Mix - Share of Slovene,Director Experience - Nationality Mix - Share of South African,Director Experience - Nationality Mix - Share of South Korean,Director Experience - Nationality Mix - Share of Spanish,Director Experience - Nationality Mix - Share of Sri Lankan,Director Experience - Nationality Mix - Share of Swedish,Director Exp

# Additional 4: Get CEO and CFO education variables and compensation ratios

In [40]:
colstokeep_identifiers = ['CompanyID*', 'Year']
colstokeep_variables = ['Ratios - Bonus/ (Bonus&Salary)','Ratios - Equity Linked/ Total', 'Ratios - Performance/ Total', 
                        'Ratios - %Change from Last Period', 'Ratios - Wealth Delta',
                        'Highest Ranked Degree - Institution Name', 'Highest Ranked Degree - InstitutionID*', 
                        'Highest Ranked Degree - Qualification', 'Highest Ranked Degree - Country']

In [41]:
def process_ceo_and_cfo_datasets(df, ceo_or_cfo, colstokeep_identifiers, colstokeep_variables):
    df = df[df[ceo_or_cfo] == True]
    df_groupby = df.groupby(['CompanyID*', 'Year']).first().reset_index()
    df_groupby = df_groupby[colstokeep_identifiers + colstokeep_variables]
    colstokeep_variables_renamed = [ceo_or_cfo + ' - ' + var for var in colstokeep_variables]
    colstokeep_variables_dict = {key: value for key, value in zip(colstokeep_variables, colstokeep_variables_renamed)}
    df_groupby = df_groupby.rename(columns=colstokeep_variables_dict)
    return df_groupby

In [42]:
df_ceo_groupby = process_ceo_and_cfo_datasets(df, 'CEO', colstokeep_identifiers, colstokeep_variables)
df_cfo_groupby = process_ceo_and_cfo_datasets(df, 'CFO', colstokeep_identifiers, colstokeep_variables)
df_ceo_groupby

,CompanyID*,Year,CEO - Ratios - Bonus/ (Bonus&Salary),CEO - Ratios - Equity Linked/ Total,CEO - Ratios - Performance/ Total,CEO - Ratios - %Change from Last Period,CEO - Ratios - Wealth Delta,CEO - Highest Ranked Degree - Institution Name,CEO - Highest Ranked Degree - InstitutionID*,CEO - Highest Ranked Degree - Qualification,CEO - Highest Ranked Degree - Country
0,386710,2019,NaN,NaN,NaN,NaN,NaN,Colorado School of Mines (CSM),727322518.0,BSc,United States
1,386710,2020,NaN,NaN,NaN,NaN,NaN,Colorado School of Mines (CSM),727322518.0,BSc,United States
2,501657,2006,NaN,NaN,NaN,NaN,NaN,University of Colorado,756236896.0,MBA,United States
3,501657,2007,NaN,NaN,NaN,NaN,NaN,University of Colorado,756236896.0,MBA,United States
4,501657,2008,NaN,NaN,NaN,NaN,NaN,University of Colorado,756236896.0,MBA,United States
...,...,...,...,...,...,...,...,...,...,...,...
10294,305214912690,2020,NaN,NaN,NaN,NaN,NaN,Texas Tech University,745125595.0,BSc,United States
10295,305299712697,2017,NaN,NaN,NaN,NaN,NaN,Universitetet i Oslo (University of Oslo),750706282.0,Candidate of Law (Cand Jur),Norway
10296,305299712697,2018,NaN,NaN,NaN,NaN,NaN,Universitetet i Oslo (University of Oslo),750706282.0,Candidate of Law (Cand Jur),Norway
10297,305299712697,2019,NaN,NaN,NaN,NaN,NaN,Uppsala Universitet (Uppsala University),755666837.0,MSc,Sweden


In [43]:
# Combine the ceo and cfo datasets
df_ceo_cfo = df_ceo_groupby.merge(df_cfo_groupby, on=['CompanyID*', 'Year'], how='outer')
df_ceo_cfo

,CompanyID*,Year,CEO - Ratios - Bonus/ (Bonus&Salary),CEO - Ratios - Equity Linked/ Total,CEO - Ratios - Performance/ Total,CEO - Ratios - %Change from Last Period,CEO - Ratios - Wealth Delta,CEO - Highest Ranked Degree - Institution Name,CEO - Highest Ranked Degree - InstitutionID*,CEO - Highest Ranked Degree - Qualification,CEO - Highest Ranked Degree - Country,CFO - Ratios - Bonus/ (Bonus&Salary),CFO - Ratios - Equity Linked/ Total,CFO - Ratios - Performance/ Total,CFO - Ratios - %Change from Last Period,CFO - Ratios - Wealth Delta,CFO - Highest Ranked Degree - Institution Name,CFO - Highest Ranked Degree - InstitutionID*,CFO - Highest Ranked Degree - Qualification,CFO - Highest Ranked Degree - Country
0,386710,2019,NaN,NaN,NaN,NaN,NaN,Colorado School of Mines (CSM),727322518.0,BSc,United States,NaN,NaN,NaN,NaN,NaN,Simon Fraser University,7.428453e+08,BS,Canada
1,386710,2020,NaN,NaN,NaN,NaN,NaN,Colorado School of Mines (CSM),727322518.0,BSc,United States,NaN,NaN,NaN,NaN,NaN,Simon Fraser University,7.428453e+08,BS,Canada
2,501657,2006,NaN,NaN,NaN,NaN,NaN,University of Colorado,756236896.0,MBA,United States,NaN,NaN,NaN,NaN,NaN,St Francis College,1.041421e+10,Graduated,United States
3,501657,2007,NaN,NaN,NaN,NaN,NaN,University of Colorado,756236896.0,MBA,United States,NaN,NaN,NaN,NaN,NaN,St Francis College,1.041421e+10,Graduated,United States
4,501657,2008,NaN,NaN,NaN,NaN,NaN,University of Colorado,756236896.0,MBA,United States,NaN,NaN,NaN,NaN,NaN,St Francis College,1.041421e+10,Graduated,United States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19177,297608412150,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hofstra University,7.321836e+08,MBA,United States
19178,297608412150,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hofstra University,7.321836e+08,MBA,United States
19179,297608412150,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hofstra University,7.321836e+08,MBA,United States
19180,307364612844,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,University of Texas at Austin (UT Austin),7.527765e+08,Masters Degree,United States


# Additional 5: Get Company Details

In [44]:
df_companydetails = pd.read_csv(r"Company Details.csv", low_memory=False)
df_companydetails

,CIK Code,HOAddress2,HOAddress1,Company Name,HOAddress4,Auditors,Latest AR,HO TelNumber,Bankers,Index,Ticker,Market Cap,ISIN,CompanyID*,HO FaxNumber,Sector,HOAddress5,HOCountryName,HO URL,HOAddress3,inputfilestem,region,mgmt_type,workbook_name,fragment_number,sheet_name
0,NaN,Saint Helier,12 Castle Street,3I INFRASTRUCTURE PLC (3i Infrastructure Ltd p...,NaN,Deloitte LLP (Deloitte & Touche LLP prior to 1...,Mar 2020,+44 (0)1534 711444,Citibank Europe PLC,FTSE 250,3IN,3040,JE00BF5FX167,9811997164,+44 (0)1534 609333,Investment Companies,JE2 3RT,Jersey,http://www.3i-infrastructure.com/,Channel Islands,Europe - Company Details,Europe,Director,Company Details,NaN,NaN
1,NaN,Aarhus C,Christian Filtenborgs Plads 1,A/S SCHOUW & CO,Midtjylland,Ernst & Young Godkendt Revisionspartnerselskab...,Dec 2020,+45 86 11 22 22,NaN,NaN,SCHO,2321,DK0010253921,3552310896,+45 86 11 33 22,Diversified Industrials,DK-8000,Denmark,https://www.schouw.dk/en/home/,NaN,Europe - Company Details,Europe,Director,Company Details,NaN,NaN
2,NaN,NaN,Via Lamarmora 230,A2A SPA,Brescia,EY SpA (Formerly known as Reconta Ernst & Youn...,Dec 2019,+39 030 35531,NaN,NaN,A2A,4883,IT0001233417,24982801,+39 030 355 3204,Electricity,25124,Italy,https://www.a2a.eu/en,NaN,Europe - Company Details,Europe,Director,Company Details,NaN,NaN
3,NaN,Wiesbaden,Paulinenstrasse 15,AAREAL BANK AG,Hessen,KPMG AG Wirtschaftsprüfungsgesellschaft (Forme...,Dec 2019,+49 611 348 0,NaN,NaN,ARL,1476,DE0005408116,947432828,+49 611 348 25 49,Banks,D-65189,Germany,https://www.aareal-bank.com/en/,NaN,Europe - Company Details,Europe,Director,Company Details,NaN,NaN
4,1091587.0,NaN,Affolternstrasse 44,ABB LTD,Zurich,KPMG AG,Dec 2020,+41 (0)43 317 7111,Citibank NA (Formerly known as First National ...,"OMX Stockholm 30, SMI , Stoxx Europe 50","ABB, ABBN",57423,CH0012221716,14501792,+41 (0)43 317 4420,Engineering & Machinery,CH-8050,Switzerland,https://global.abb/group/en,NaN,Europe - Company Details,Europe,Director,Company Details,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3310,1451554.0,NaN,1 Bedford Avenue,WILLIAM HILL PLC,London,Deloitte LLP (Deloitte & Touche LLP prior to 1...,Dec 2020,+44 (0) 20 8918 3600,NaN,NaN,WMH,2892,GB0031698896,837912719,+44 (0) 20 8918 3775,Leisure & Hotels,WC1B 3AU,United Kingdom - England,https://www.williamhillplc.com/,NaN,UK - Company Details,UK,Director,Company Details,NaN,NaN
3311,NaN,Kennington Park Business Centre,Canterbury Court,WORKSPACE GROUP PLC (London Industrial PLC pri...,London,KPMG LLP,Mar 2020,+44 (0) 20 7138 3300,NatWest Markets PLC (Royal Bank of Scotland PL...,FTSE 250,WKP,1472,GB00B67G5X01,428697608,+44 (0) 20 7247 0157,Real Estate,SW9 6DE,United Kingdom - England,https://www.workspace.co.uk/,1-3 Brixton Road,UK - Company Details,UK,Director,Company Details,NaN,NaN
3312,NaN,25 Walbrook,The Walbrook Building,WORLDPAY GROUP PLC (De-listed 01/2018),London,NaN,Jan 2018,NaN,NaN,NaN,WPG,0,GB00BYYK2V80,24259147400,NaN,Business Services,EC4N 8AF,United Kingdom - England,http://www.worldpay.com/global,NaN,UK - Company Details,UK,Director,Company Details,NaN,NaN
3313,806968.0,18 Upper Ground,Sea Containers,WPP PLC (WPP Group prior to 11/2008),London,Deloitte LLP (Deloitte & Touche LLP prior to 1...,Dec 2019,+44 (0) 20 7408 2204,Citibank NA (Formerly known as First National ...,FTSE 100,WPP,11568,JE00B8KF9B49,429577695,+44 (0) 20 7493 6819,Media & Entertainment,SE1 9GL,United Kingdom - England,https://www.wpp.com/,NaN,UK - Company Details,UK,Director,Company Details,NaN,NaN


In [45]:
df_companydetails.columns

Index(['CIK Code', 'HOAddress2', 'HOAddress1', 'Company Name', 'HOAddress4',
       'Auditors', 'Latest AR', 'HO TelNumber', 'Bankers', 'Index', 'Ticker',
       'Market Cap', 'ISIN', 'CompanyID*', 'HO FaxNumber', 'Sector',
       'HOAddress5', 'HOCountryName', 'HO URL', 'HOAddress3', 'inputfilestem',
       'region', 'mgmt_type', 'workbook_name', 'fragment_number',
       'sheet_name'],
      dtype='object')

In [46]:
df_companydetails = df_companydetails.drop_duplicates(['CompanyID*']).drop(columns=['inputfilestem', 'region', 'mgmt_type', 'workbook_name', 'fragment_number', 'sheet_name'])
df_companydetails.to_csv(r'C:\Users\jasonjia\Dropbox\projects\manager_and_board_characteristics\Board Charac_Jason\output\03_get_firm_and_manager_level_datasets\Company Details.csv', index=False)
df_companydetails

,CIK Code,HOAddress2,HOAddress1,Company Name,HOAddress4,Auditors,Latest AR,HO TelNumber,Bankers,Index,Ticker,Market Cap,ISIN,CompanyID*,HO FaxNumber,Sector,HOAddress5,HOCountryName,HO URL,HOAddress3
0,NaN,Saint Helier,12 Castle Street,3I INFRASTRUCTURE PLC (3i Infrastructure Ltd p...,NaN,Deloitte LLP (Deloitte & Touche LLP prior to 1...,Mar 2020,+44 (0)1534 711444,Citibank Europe PLC,FTSE 250,3IN,3040,JE00BF5FX167,9811997164,+44 (0)1534 609333,Investment Companies,JE2 3RT,Jersey,http://www.3i-infrastructure.com/,Channel Islands
1,NaN,Aarhus C,Christian Filtenborgs Plads 1,A/S SCHOUW & CO,Midtjylland,Ernst & Young Godkendt Revisionspartnerselskab...,Dec 2020,+45 86 11 22 22,NaN,NaN,SCHO,2321,DK0010253921,3552310896,+45 86 11 33 22,Diversified Industrials,DK-8000,Denmark,https://www.schouw.dk/en/home/,NaN
2,NaN,NaN,Via Lamarmora 230,A2A SPA,Brescia,EY SpA (Formerly known as Reconta Ernst & Youn...,Dec 2019,+39 030 35531,NaN,NaN,A2A,4883,IT0001233417,24982801,+39 030 355 3204,Electricity,25124,Italy,https://www.a2a.eu/en,NaN
3,NaN,Wiesbaden,Paulinenstrasse 15,AAREAL BANK AG,Hessen,KPMG AG Wirtschaftsprüfungsgesellschaft (Forme...,Dec 2019,+49 611 348 0,NaN,NaN,ARL,1476,DE0005408116,947432828,+49 611 348 25 49,Banks,D-65189,Germany,https://www.aareal-bank.com/en/,NaN
4,1091587.0,NaN,Affolternstrasse 44,ABB LTD,Zurich,KPMG AG,Dec 2020,+41 (0)43 317 7111,Citibank NA (Formerly known as First National ...,"OMX Stockholm 30, SMI , Stoxx Europe 50","ABB, ABBN",57423,CH0012221716,14501792,+41 (0)43 317 4420,Engineering & Machinery,CH-8050,Switzerland,https://global.abb/group/en,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3310,1451554.0,NaN,1 Bedford Avenue,WILLIAM HILL PLC,London,Deloitte LLP (Deloitte & Touche LLP prior to 1...,Dec 2020,+44 (0) 20 8918 3600,NaN,NaN,WMH,2892,GB0031698896,837912719,+44 (0) 20 8918 3775,Leisure & Hotels,WC1B 3AU,United Kingdom - England,https://www.williamhillplc.com/,NaN
3311,NaN,Kennington Park Business Centre,Canterbury Court,WORKSPACE GROUP PLC (London Industrial PLC pri...,London,KPMG LLP,Mar 2020,+44 (0) 20 7138 3300,NatWest Markets PLC (Royal Bank of Scotland PL...,FTSE 250,WKP,1472,GB00B67G5X01,428697608,+44 (0) 20 7247 0157,Real Estate,SW9 6DE,United Kingdom - England,https://www.workspace.co.uk/,1-3 Brixton Road
3312,NaN,25 Walbrook,The Walbrook Building,WORLDPAY GROUP PLC (De-listed 01/2018),London,NaN,Jan 2018,NaN,NaN,NaN,WPG,0,GB00BYYK2V80,24259147400,NaN,Business Services,EC4N 8AF,United Kingdom - England,http://www.worldpay.com/global,NaN
3313,806968.0,18 Upper Ground,Sea Containers,WPP PLC (WPP Group prior to 11/2008),London,Deloitte LLP (Deloitte & Touche LLP prior to 1...,Dec 2019,+44 (0) 20 7408 2204,Citibank NA (Formerly known as First National ...,FTSE 100,WPP,11568,JE00B8KF9B49,429577695,+44 (0) 20 7493 6819,Media & Entertainment,SE1 9GL,United Kingdom - England,https://www.wpp.com/,NaN


# Final: Merge df with the additional variables

In [47]:
df_merge = df_groupby.merge(df_charac_gender, how='left', on=['CompanyID*', 'Year'])
df_merge = df_merge.merge(df_charac_nationality, how='left', on=['CompanyID*', 'Year'])
df_merge = df_merge.merge(df_ceo_cfo, how='left', on=['CompanyID*', 'Year'])
df_merge = df_merge.merge(df_companydetails, how='inner', on=['CompanyID*', 'ISIN', 'Sector', 'Company Name'])
df_merge

,Year,CompanyID*,Company Name,Sector,Country,ISIN,region,Characteristics of Roles - Director Network Size,Characteristics of Roles - Time to Retirement,Characteristics of Roles - Time in Role,Characteristics of Roles - Time on Board,Characteristics of Roles - Time in Company,Director Experience - Total Number of Quoted Boards to Date,Director Experience - Total Number of Private Boards to Date,Director Experience - Total Number of Other Boards to Date,Director Experience - Total Number of Quoted Current Boards,Director Experience - Total Number of Private Current Boards,Director Experience - Total Number of Other Current Boards,Director Experience - Avg. Yrs on Other Quoted Boards,Director Experience - Age (Yrs),Director Experience - Number of Qualifications,Ratios - Bonus/ (Bonus&Salary),Ratios - Equity Linked/ Total,Ratios - Performance/ Total,Ratios - %Change from Last Period,Ratios - Wealth Delta,Director Count Totals - Number of Independent NED with past CFO/FD role,Annual Direct Compensation - Salary,Annual Direct Compensation - Bonus,Annual Direct Compensation - D.C Pension,Annual Direct Compensation - Other,Annual Direct Compensation - Total Salary+Bonus,Annual Direct Compensation - Total Inc. D.C. Pension & Other,Annual- Equity Linked Options - Shares,Annual- Equity Linked Options - LTIPS(max),Annual- Equity Linked Options - Intrinsic Options (excercisable),Annual- Equity Linked Options - Intrinsic Options (unexercisable),Annual- Equity Linked Options - Estimated Options (unexcercisable),Annual- Equity Linked Options - Estimated Options (exercisable),Annual- Equity Linked Options - Share Price,...,Director Experience - Nationality Mix - Share of Ugandan,Director Experience - Nationality Mix - Share of Ukrainian,Director Experience - Nationality Mix - Share of Uruguayan,Director Experience - Nationality Mix - Share of Venezuelan,Director Experience - Nationality Mix - Share of Zambian,Director Experience - Nationality Mix - Share of Zimbabwean,CEO - Ratios - Bonus/ (Bonus&Salary),CEO - Ratios - Equity Linked/ Total,CEO - Ratios - Performance/ Total,CEO - Ratios - %Change from Last Period,CEO - Ratios - Wealth Delta,CEO - Highest Ranked Degree - Institution Name,CEO - Highest Ranked Degree - InstitutionID*,CEO - Highest Ranked Degree - Qualification,CEO - Highest Ranked Degree - Country,CFO - Ratios - Bonus/ (Bonus&Salary),CFO - Ratios - Equity Linked/ Total,CFO - Ratios - Performance/ Total,CFO - Ratios - %Change from Last Period,CFO - Ratios - Wealth Delta,CFO - Highest Ranked Degree - Institution Name,CFO - Highest Ranked Degree - InstitutionID*,CFO - Highest Ranked Degree - Qualification,CFO - Highest Ranked Degree - Country,CIK Code,HOAddress2,HOAddress1,HOAddress4,Auditors,Latest AR,HO TelNumber,Bankers,Index,Ticker,Market Cap,HO FaxNumber,HOAddress5,HOCountryName,HO URL,HOAddress3
0,2002,270810,MDU RESOURCES GROUP INC,Utilities - Other,United States,US5526901096,None,654.333333,7.681818,8.225000,10.500000,12.483333,1.600000,4.777778,1.000000,1.400000,1.714286,1.0,3.090000,60.363636,2.416667,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67716.0,Post Office Box 5650,1200 West Century Avenue,"North Dakota, ND",Deloitte & Touche LLP,Dec 2020,+1 701 530 1000,NaN,S&P MID CAP 400,MDU,6347,+1 866 760 4852,58506-5650,United States,https://www.mdu.com/,Bismarck
1,2003,270810,MDU RESOURCES GROUP INC,Utilities - Other,United States,US5526901096,None,676.882353,9.283333,6.984615,8.690909,10.953846,1.727273,4.700000,1.000000,1.363636,1.857143,1.0,4.036364,58.916667,2.307692,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67716.0,Post Office Box 5650,1200 West Century Avenue,"North Dakota, ND",Deloitte & Touche LLP,Dec 2020,+1 701 530 1000,NaN,S&P MID CAP 400,MDU,6347,+1 866

# Cleanup df and save final df into csv

In [48]:
outputfilepath = Path(r"C:\Users\jasonjia\Dropbox\projects\manager_and_board_characteristics\Board Charac_Jason\output\03_get_firm_and_manager_level_datasets\boardex_firm_level_dataset.csv")
df_merge.to_csv(outputfilepath, index=False)